In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp


from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix
from scipy.sparse.linalg import spsolve

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

#import pymc as pm
#import arviz as az

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

#import pyswarms as ps

######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class StockData:
    def __init__(self, ticker, start, end, column):
        self.ticker = ticker
        self.start = start
        self.end = end
        self.df = None
        self.column=column

    def fetch_data(self):

        #######################################################################################################
        # yfinance
        df = yf.download(self.ticker, start=self.start, end=self.end)
        df.columns = df.columns.droplevel(1)
        df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
        df.index.name = 'Date' 
        self.df = df
       
        '''
        # Read XLSX and parse the 'Date' column as datetime
        filepath = '/content/drive/My Drive/LEARN/Finance/quantitative finance/Forecasting-stocks/ICICIBANK_NSE_alltime.xlsx'
        sheet = 'ICICIBANK_NSE_alltime'
        df = pd.read_excel(filepath, sheet_name=sheet, skiprows=[1, 2])

        df.set_index('Price', inplace=True)
        self.df = df[df.index >= self.start].copy()
        '''
        
        print (self.df.head())
        print (self.df.tail())
        ################################################################################################################

        self.df = self.df.sort_index()
        self.df = self.df.dropna()
        self.df.info()

        #PLOT
        ax = self.df.drop(columns=["Volume"]).plot(figsize=(12, 6), title=self.ticker + "Stock Prices")
        ax.set_ylabel("Stock Price")
        ax2 = ax.twinx()
        ax2.set_ylabel("Volume")
        ax2.plot(self.df.index, self.df["Volume"], color="gray", alpha=0.5, linestyle="dashed", label="Volume")
        ax2.legend(loc="upper left")
        plt.show()

        self.df.columns = self.df.columns.get_level_values(0)
        new_df = self.df[[self.column]].copy()
        new_df.info()

        #self.CAGR(new_df)
        return new_df

    def download_option_data(self):

        obs_price=[]
        strike = []
        option_type=[]
        IV = []

        expirations_values=[]

        ###############################################################################################################
        # Read CSV and parse the 'Date' column as datetime
        filepath = '/content/drive/My Drive/LEARN/Finance/quantitative finance/Pricing-derivatives/vanilla-stock-options/IBN_option_chain.xlsx'
        dff = pd.read_excel(filepath, sheet_name='Sheet1')
        # Ensure expiration column is datetime
        dff['expiration'] = pd.to_datetime(dff['expiration'])
        print (dff.info())

        # Iterate over each expiration and display a few rows
        #for ii in range(len(dff)):
        for ii in [0, 1, len(dff) - 2, len(dff) - 1]:
            option_type.append(dff['Type'].iloc[ii])
            obs_price.append(dff['lastPrice'].iloc[ii])
            strike.append(dff['strike'].iloc[ii])
            IV.append(dff['impliedVolatility'].iloc[ii])
            expirations_values.append(dff['expiration'].iloc[ii])


        ''' yfinance
        # Create a Ticker object
        stock = yf.Ticker(self.ticker)

        # Get available expiration dates
        expirations = stock.options
        print("Available Expirations:", expirations)
        for i in range(len(expirations)):
            opt_chain = stock.option_chain(expirations[i])
            calls = opt_chain.calls
            puts = opt_chain.puts

            if  i==0:
               print (calls.head())
               print (puts.head())

            # Sort by strike price to get the first (lowest)
            calls_sorted = calls.sort_values("strike")
            puts_sorted = puts.sort_values("strike")

            for ii in range(len(calls_sorted.index)):
                option_type.append('call')
                obs_price.append(calls_sorted['lastPrice'].iloc[ii])
                strike.append(calls_sorted['strike'].iloc[ii])
                IV.append(calls_sorted['impliedVolatility'].iloc[ii])
                expirations_values.append(expirations[i])


            for ii in range(len(puts_sorted.index)):
                option_type.append('put')
                obs_price.append(puts_sorted['lastPrice'].iloc[ii])
                strike.append(puts_sorted['strike'].iloc[ii])
                IV.append(puts_sorted['impliedVolatility'].iloc[ii])
                expirations_values.append(expirations[i])
        '''
        #####################################################################################################

        records = pd.DataFrame({
            'expirations': pd.Series(expirations_values),
            'option_type': pd.Series(option_type, dtype='str'),
            'strike': pd.Series(strike, dtype='float'),
            'obs_price': pd.Series(obs_price, dtype='float'),
            'IV': pd.Series(IV, dtype='float')
        })

        print (records)
        print (records.info())
        return np.array(expirations_values), np.array(strike), np.array(option_type), np.array(obs_price), records

    def exotic_option_data(self):
      asian_option = []
      barrier_option = []
      basket_option = []
      binary_option = [] #digital
      chooser_option = []
      compund_option = []
      extentible_option = []
      lookback_option = []
      spread_option = []
      range_option =[]


      shout_option=[]
      cliquet_option = []
      quanto_option = []
      rainbow_option =[]

      forward_start_option =[]
      reset_option = [] #ratchet
      timer_option = []
      rebate_barrier_option =[]
      corridor_option =[]
      knock_out_fprward_option =[]
      soft_barrier_option =[]
      volatility_swap_option =[]
      extentible_option =[]
      cancelable_option = []




    #change to OOP later
    def CAGR(self, df):
        # Ensure the DataFrame index is datetime
        df = df.sort_index()
        if not isinstance(df.index, pd.DatetimeIndex):
            raise ValueError("DataFrame index must be a DatetimeIndex.")

        yearly_cagr = {}
        cumulative_cagr = {}

        years = sorted(df.index.year.unique())

        # Use the second year as the start to avoid incomplete initial data
        start_year = years[0]
        start_date = df[df.index.year == start_year].index[0]
        start_value = df.loc[start_date].values[0]

        for year in years:  # Skip the first year
            # Yearly CAGR
            yearly_df = df[df.index.year == year]
            start = yearly_df.iloc[0].values[0]
            end = yearly_df.iloc[-1].values[0]
            days = (yearly_df.index[-1] - yearly_df.index[0]).days
            if days > 0:
                yearly_cagr[year] = (end / start) ** (365.25 / days) - 1

            # Cumulative CAGR from start_year to current year
            end_value = df[df.index.year <= year].iloc[-1].values[0]
            total_days = (df[df.index.year <= year].index[-1] - start_date).days
            if total_days > 0:
                cumulative_cagr[year] = (end_value / start_value) ** (365.25 / total_days) - 1

        # Plot
        plt.figure(figsize=(12, 6))

        # Yearly CAGR line
        plt.plot(yearly_cagr.keys(), [v * 100 for v in yearly_cagr.values()],
                marker='o', label='Yearly CAGR')

        # Cumulative CAGR line
        cumulative_values = [v * 100 for v in cumulative_cagr.values()]
        plt.plot(cumulative_cagr.keys(), cumulative_values,
                marker='s', linestyle='--', label='Cumulative CAGR')

        # Annotate cumulative CAGR values
        for year, val in cumulative_cagr.items():
            plt.text(year, val * 100 + 0.5, f"{val * 100:.2f}%", ha='center', fontsize=8, color='blue')

        plt.axhline(0, color='black', linewidth=0.5)
        plt.title("Yearly vs Cumulative CAGR of NIFTY Close Prices")
        plt.xlabel("Year")
        plt.ylabel("CAGR (%)")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()